# Suspect of too low accuracy in Lesion model
Vocabulary: 
- boardcasting: when adding tensor with different dimension, for example input_p (dim = [batch_size, 250]) and bias_p (dim = [1, 250]), bias_p will automatically "boardcast" to (batch_size, 250) during the elementwise addition

When input information have not reach a layer, it will assume the first axis (batch_size) = 1 
however, the batch_size won't matter... since tf.matmul batch_axis is independent with each other...  


In [ ]:
import tensorflow as tf
import pandas as pd
import os 
import meta, modeling, data_wrangling, evaluate
from importlib import reload
reload(modeling)


def init(code_name):
    cfg = meta.ModelConfig.from_json(
        os.path.join("models", code_name, "model_config.json")
    )
    cfg.output_ticks = 13
    model = modeling.MyModel(cfg, batch_size_override=100)
    model.load_weights(cfg.saved_weights_fstring.format(epoch=300))
    return cfg, model


cfg, model = init("Refrac_5M")
train100 = data_wrangling.load_testset('dataset/testsets/train_r100.pkl.gz')



# Load testset and evaluate an epoch
## y_pred[task][output_name][timetick][tensor]

In [ ]:
model.set_active_task("triangle")
y_pred = model([train100['ort']] * cfg.n_timesteps)

print(f"Output keys: {y_pred.keys()}")
print(f"Weight tensors: {[w.name for w in model.weights]}")


In [ ]:
# Collecting SEM layer useful values

idx = train100['item'].index("fit")
df_dict = {}

# Time invariant elements
df_dict['target'] = train100['sem'][idx, :].numpy()
df_dict['bias'] = [w.numpy() for w in model.weights if w.name == 'my_model/bias_s:0'][0]
df_dict['act_sem'] = y_pred['sem'][-1,idx,:].numpy() # Last tick activation

NAME_MAP = {'input_hps_hs':'PS', 'input_css_cs':'CS', 'input_sem_ss':'SS', 'input_hos_hs':'OS', 'sem':'act_sem'}
# Time varying elements


In [ ]:

df = pd.DataFrame()
for input_name in ('input_hps_hs', 'input_css_cs', 'input_sem_ss', 'input_hos_hs', 'sem'):
    this_input_df = pd.DataFrame()
    for t in range(13):
        df_dict = {}
        df_dict[NAME_MAP[input_name]] = y_pred[input_name][t,idx,:].numpy()
        this_step_df = pd.DataFrame.from_dict(df_dict)
        this_step_df['timetick'] = t
        this_step_df['unit'] = this_step_df.index
        this_input_df = pd.concat([this_input_df, this_step_df])
    df = pd.concat([df, this_input_df], axis=1)

df[['timetick', 'unit', 'act_sem', 'OS', 'SS', 'CS', 'PS']]


In [ ]:
df

In [ ]:
pdf = df[['target', 'bias', 'act_12']].copy()
pdf['unit'] = pdf.index

In [ ]:
import altair as alt

unit_selection = alt.selection_single()

point = alt.Chart(pdf).mark_point().encode(
    x='bias:Q',
    y='act_12:Q',
    color="target:N",
    opacity=alt.condition(unit_selection, alt.value(1.), alt.value(0.2)),
    tooltip=['unit', 'target', 'act_12', 'bias']
).add_selection(unit_selection)

vline = alt.Chart(pd.DataFrame({'x': [0.]})).mark_rule(color='red', strokeWidth=3).encode(x='x:Q')
vline + point

In [ ]:
import altair as alt

def plot_activation_by_target(df):
    sdf = df[['target', 'bias', 'act_12']]
    sdf = sdf.pivot_table(column='target').reset_index()

    plot_activation = (
        alt.Chart(sdf).mark_point().encode(
        x=alt.X("1:Q", scale=alt.Scale(domain=(0,1)), title="Activation for target node = 1"),
        y=alt.Y("0:Q", scale=alt.Scale(reverse=True, domain=(0,1)), title="Activation for target node = 0"),
        color="task:N",
        detail="word:N",
        column=alt.Column("acc:N", title="Prediction accuracy"),
        tooltip=['word', 'acc', 'sse', 'act0', 'act1' ]
        )
        .transform_filter(interval_epoch)
        .transform_filter(t)
        .properties(title=f"{output} diagnostic")
    )

    return epoch_selection & timetick_sel & plot_activation


In [ ]:
train100['sem'][idx, :].numpy()

In [ ]:
y_pred['sem'][,idx,:].numpy()

In [ ]:
pd.DataFrame.from_dict(df_dict)

In [ ]:
bias_s = 

In [ ]:
bias_s

In [ ]:
[print(w.index) for w in train100['item'] if w == 'close']

In [ ]:
# Input from O to S, more precisely: tf.matmul(hos, w_hos_hs)  
os = y_pred['input_hos_hs']

# Input from P to S, more precisely: tf.matmul(hps, w_hps_hs)
ps = y_pred['input_hps_hs']

# Input from C to S, more precisely: tf.matmul(css, w_cs)
cs = y_pred['input_css_cs']

# Input from S to S, more precisely: tf.matmul(sem, w_ss)
ss = y_pred['input_sem_ss']

# Input from O to P, more precisely: tf.matmul(hop, w_hop_hp)
op = y_pred['input_hop_hp']

print(f"OS shape:{os.shape}")
print(f"PS shape:{ps.shape}")
print(f"CS shape:{cs.shape}")
print(f"SS shape:{ss.shape}")
print(f"OP shape:{op.shape}")

In [ ]:
def mean_unit_input(tensor, mask=None):
    """Mask with output target, reduce sum on items (axis1), mean on units (axis2)"""
    if mask is not None:
        tensor = tf.multiply(tensor, mask)
    return tf.reduce_mean(tf.reduce_mean(tensor, axis=-1), axis=-1)



In [ ]:

results_dict = {k: mean_unit_input(globals()[k], train100['sem']) for k in ('os', 'ps', 'cs', 'ss')}
results_dict['op'] = mean_unit_input(op, train100['pho'])
dol_df = pd.DataFrame.from_dict(results_dict)
dol_df.plot(title="one target")

In [ ]:
results_dict = {k: mean_unit_input(globals()[k], 1-train100['sem']) for k in ('os', 'ps', 'cs', 'ss')}
results_dict['op'] = mean_unit_input(op, 1-train100['pho'])
dol_df = pd.DataFrame.from_dict(results_dict)
dol_df.plot(title="zero target")

In [ ]:
bias_s = [w for w in model.weights if w.name == "my_model/bias_s:0"]
bias_p = [w for w in model.weights if w.name == "my_model/bias_p:0"]
print(f"mean bias S: {tf.reduce_mean(bias_s)}; P: {tf.reduce_mean(bias_p)}")

- Kind of neutral P bias
- Positive bias in S??? is counter int

In [ ]:
tf.